In [ ]:
import json
import time
import sys

from datetime import datetime
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


This Notebook is used to getting preview_url from Spotify using Spotify developer account.

Please note that fetching previews will take a big amount of time. It depends on your network connection and precessing power of the machine you are running the notebook on

Please change below from your Spotify developer account

In [ ]:
#Log in using Client ID and secret given from your spotify developer account
cid = "5efa6b4c52b0498ba99ca31f0974dfea"
secret = "e5b16d1d00f94996ad0f8b9150d4d3e4"

In [ ]:
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)
sp.trace = True

In [ ]:
# Read the CSV file
df = pd.read_csv('/Users/sasha.cures/Documents/GitHub/group-coursework-machine-learners/notebooks/CNN/sp_popular_tracks_ids.csv')

# Get all the values from the CSV column except the header
all_values = df.iloc[1:, 0].astype(str).values.tolist()

In [ ]:
# Create a list to store the chunks
chunks = []

In [ ]:
# Define the chunk size and string size
chunk_size = 5000
string_size = 50

In [ ]:
# Initialize counters
current_chunk = []
current_string = [] 

Sort data into chunks

In [ ]:
for value in all_values:
    current_string.append(value)

    # Check if the current string size has reached 50
    if len(current_string) == string_size:
        current_chunk.append(','.join(current_string))
        current_string = []

    # Check if the current chunk size has reached 5000
    if len(current_chunk) == chunk_size:
        chunks.append(current_chunk)
        current_chunk = []

# Append any remaining values
if current_chunk:
    chunks.append(current_chunk)

Create function for fetching preview url from Spotify

In [ ]:
def fetch_preview_urls(chunks, chunk_index):
    if chunk_index < 0 or chunk_index >= len(chunks):
        print("Invalid chunk index")
        return [], []

    preview_urls = []
    no_preview_track_ids = []
    chunk = chunks[chunk_index]

    for track_ids_string in chunk:
        track_ids = track_ids_string.split(',')
        
        try:
            tracks_info = sp.tracks(tracks=track_ids)
            
            for track in tracks_info['tracks']:
                preview_url = track.get('preview_url')
                track_id = track.get('id')
                if preview_url:
                    preview_urls.append((preview_url, track_id))  # Store as a tuple
                else:
                    no_preview_track_ids.append(track_id)

        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429:
                wait_time = int(e.response.headers.get('Retry-After', 1))
                print(f"Rate limit reached. Waiting for {wait_time} seconds.")
                time.sleep(wait_time)
            else:
                print(f"HTTP error: {str(e)}")
        except Exception as e:
            print(f"Error: {str(e)}")

    return preview_urls, no_preview_track_ids


Specify desired chunk of data to iterate over

In [ ]:
desired_chunk_index = 2 # Starts from 0

Iterate through chunk fetching previews from Spotify

In [ ]:
if 0 <= desired_chunk_index < len(chunks):
    # Fetch preview URLs and track IDs without preview URLs
    preview_urls, no_preview_track_ids = fetch_preview_urls(chunks, desired_chunk_index)

    preview_urls, track_ids = zip(*preview_urls)
    
    # Create DataFrames
    preview_df = pd.DataFrame({'Preview_URL': preview_urls, 'Track_ID': track_ids})
    no_preview_df = pd.DataFrame({'No_Preview_Track_ID': no_preview_track_ids})

    # Save the DataFrames to CSV files
    preview_df.to_csv('./Spectograms_images/preview_urls_'+str(desired_chunk_index)+'.csv', index=False)
    no_preview_df.to_csv('./Spectograms_images/no_preview_track_ids_'+str(desired_chunk_index)+'.csv', index=False)

    print(f'Desired Chunk {desired_chunk_index} Preview URLs saved to "preview_urls.csv"')
    print(f'Track IDs without preview URLs from Chunk {desired_chunk_index} saved to "no_preview_track_ids.csv"')
else:
    print('Invalid chunk index')